# Predict

In [96]:
import sys
sys.path.insert(0,"/home/gridsan/hwpang/Software/RMG-Py/")
sys.path.insert(0,"..")

import json
import pandas as pd

from rmgpy.data.thermo import ThermoDatabase, ThermoGroups

from tree.thermo import ThermoGroups as SIDTThermoGroups
from tree.utils import make_mol
from tree.parameters import Ts

# Load test data

In [97]:
hbi_unc_df = pd.read_csv("../data/hbi_unc.csv")

In [98]:
with open("../data/splits/random.json", "r") as f:
    train_inds, test_inds = json.load(f)

In [99]:
test_df = hbi_unc_df.loc[test_inds, :]
test_df

,radical_smiles,radical_resonance_smiles,closed_shell_smiles,closed_shell_H298 (kcal/mol),closed_shell_S298 (cal/mol/K),closed_shell_Cp300 (cal/mol/K),closed_shell_Cp400 (cal/mol/K),closed_shell_Cp500 (cal/mol/K),closed_shell_Cp600 (cal/mol/K),closed_shell_Cp800 (cal/mol/K),...,unc_HBI_H298 (kcal/mol),unc_HBI_S298 (cal/mol/K),unc_HBI_Sint298 (cal/mol/K),unc_HBI_Cp300 (cal/mol/K),unc_HBI_Cp400 (cal/mol/K),unc_HBI_Cp500 (cal/mol/K),unc_HBI_Cp600 (cal/mol/K),unc_HBI_Cp800 (cal/mol/K),unc_HBI_Cp1000 (cal/mol/K),unc_HBI_Cp1500 (cal/mol/K)
14,[O]C(=O)OC(=O)O,[O]C(=O)OC(=O)O,O=C(O)OC(=O)O,-223.695934,86.503270,24.694823,28.758100,32.040339,34.628195,38.156001,...,2.4,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
18,[O]OC(=O)OC(=O)O,[O]OC(=O)OC(=O)O,O=C(O)OC(=O)OO,-199.973245,94.255903,31.001150,34.739959,37.826260,40.355462,44.035249,...,2.4,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
25,C[C]1OC(=O)C(O)(C=O)O1,C[C]1OC(=O)C(O)(C=O)O1,CC1OC(=O)C(O)(C=O)O1,-206.467878,99.587328,33.632639,44.454340,53.571377,59.780918,67.788604,...,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
36,CC1(OO)COC(O[O])C(=O)O1,CC1(OO)COC(O[O])C(=O)O1,CC1(OO)COC(OO)C(=O)O1,-194.098719,108.407920,48.100822,61.544551,72.689006,80.724321,90.184054,...,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
49,[O]OC1(COO)COCC(=O)O1,[O]OC1(COO)COCC(=O)O1,O=C1COCC(COO)(OO)O1,-180.828929,108.787897,49.031759,62.443461,73.174034,81.237572,90.690096,...,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2968,[CH2]C#CC=C=C,C=C=[C]C=C=C,C=C=CC=C=C,96.710913,76.828171,26.525688,32.732134,37.443511,41.260269,47.258558,...,2.4,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2969,[CH2]C#CC=C=C,C=[C]C=C=C=C,C=C=C=CC=C,92.368996,78.947008,26.676979,32.843489,37.671520,41.507017,47.122008,...,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2979,C=C=C1[CH]C1=C,C=C=C1[CH]C1=C,C=C=C1CC1=C,112.767974,53.168531,24.008919,30.043266,34.942461,38.950145,44.884263,...,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2980,C=C=C1[CH]C1=C,C=[C]C1=CC1=C,C=CC1=CC1=C,118.596000,75.148273,23.377000,29.587000,34.875000,39.276000,46.044000,...,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [100]:
mols = test_df["radical_resonance_smiles"].apply(make_mol)
mols = mols.to_list()

# Load thermo database

In [101]:
def make_prediction(thermo_database, tree, mol):
    atoms = {"*": atom for atom in mol.atoms if atom.radical_electrons==1}
    return thermo_database._add_group_thermo_data(None, tree, mol, atoms)[0]

In [102]:
thermo_database = ThermoDatabase()

# Predict with empirical tree

In [107]:
empirical_tree = ThermoGroups().load("/home/gridsan/hwpang/Software/RMG-database/input/thermo/groups/radical.py", thermo_database.local_context, thermo_database.global_context)

In [108]:
thermos = [make_prediction(thermo_database, empirical_tree, mol) for mol in mols]

In [109]:
test_result_df = test_df[["radical_resonance_smiles"]]
test_result_df["HBI_H298 (kcal/mol)"] = [thermo.H298.value_si/4184 for thermo in thermos]
test_result_df["unc_HBI_H298 (kcal/mol)"] = [thermo.H298.uncertainty_si/4184 for thermo in thermos]
test_result_df["HBI_Sint298 (cal/mol/K)"] = [thermo.S298.value_si/4.184 for thermo in thermos]
test_result_df["unc_HBI_Sint298 (cal/mol/K)"] = [thermo.S298.uncertainty_si/4.184 for thermo in thermos]
for i, T in enumerate(Ts):
    test_result_df[f"HBI_Cp{T} (cal/mol/K)"] = [thermo.Cpdata.value_si[i]/4.184 for thermo in thermos]
    test_result_df[f"unc_HBI_Cp{T} (cal/mol/K)"] = [thermo.Cpdata.uncertainty_si[i]/4.184 for thermo in thermos]
test_result_df["comment"] = [thermo.comment for thermo in thermos]

/home/gridsan/hwpang/mambaforge/envs/rmg_rdmc_env_20230623_v2/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/gridsan/hwpang/mambaforge/envs/rmg_rdmc_env_20230623_v2/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/gridsan/hwpang/mambaforge/envs/rmg_rdmc_env_20230623_v2/lib/python3.7/site-packages/ipykernel_launcher.

In [110]:
test_result_df.to_csv("../data/splits/test_empirical.csv", index=False)

# Predict with SIDT tree

In [111]:
sidt_tree = SIDTThermoGroups().load("../models/split-random_run-test/tree.py", thermo_database.local_context, thermo_database.global_context)

In [112]:
thermos = [make_prediction(thermo_database, sidt_tree, mol) for mol in mols]

In [113]:
test_result_df = test_df[["radical_resonance_smiles"]]
test_result_df["HBI_H298 (kcal/mol)"] = [thermo.H298.value_si/4184 for thermo in thermos]
test_result_df["unc_HBI_H298 (kcal/mol)"] = [thermo.H298.uncertainty_si/4184 for thermo in thermos]
test_result_df["HBI_Sint298 (cal/mol/K)"] = [thermo.S298.value_si/4.184 for thermo in thermos]
test_result_df["unc_HBI_Sint298 (cal/mol/K)"] = [thermo.S298.uncertainty_si/4.184 for thermo in thermos]
for i, T in enumerate(Ts):
    test_result_df[f"HBI_Cp{T} (cal/mol/K)"] = [thermo.Cpdata.value_si[i]/4.184 for thermo in thermos]
    test_result_df[f"unc_HBI_Cp{T} (cal/mol/K)"] = [thermo.Cpdata.uncertainty_si[i]/4.184 for thermo in thermos]
test_result_df["comment"] = [thermo.comment for thermo in thermos]

/home/gridsan/hwpang/mambaforge/envs/rmg_rdmc_env_20230623_v2/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/gridsan/hwpang/mambaforge/envs/rmg_rdmc_env_20230623_v2/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/gridsan/hwpang/mambaforge/envs/rmg_rdmc_env_20230623_v2/lib/python3.7/site-packages/ipykernel_launcher.

In [114]:
split = "random"
run_name = "test"
test_result_df.to_csv(f"../models/split-{split}_run-{run_name}/test.csv", index=False)